In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import pyccl.ccllib as lib
import pyccl as ccl
import pyccl.nl_pt as pt
from astropy.table import Table
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
cosmo = ccl.Cosmology(
    Omega_c=0.22, Omega_b=0.0448, 
    h=0.71, sigma8 = 0.801, n_s= 0.963,w0=-1.00,wa=0.0, Omega_k=0.0)

Om_m = cosmo['Omega_m']
rho_crit = lib.cvar.constants.RHO_CRITICAL
Aia=1.0 #1.5,2
C2=1.0
sigma_epsilon = 0.27

In [3]:
def Epsilon1_NLA(cosmo,z,A1,rho_crit,sxx,syy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = -1*A1*5e-14*rho_crit*cosmo['Omega_m']/gz
    #e1_NLA =  - Fact  * (sxx - syy) 
    e1_NLA =  Fact  * (sxx - syy) 
    return e1_NLA

def Epsilon2_NLA(cosmo,z,A1,rho_crit,sxy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = -1*A1*5e-14*rho_crit*cosmo['Omega_m']/gz
    e2_NLA = 2 *Fact* sxy
    return e2_NLA

def Epsilon1_TT(cosmo,z,A2,rho_crit,sxx,syy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = - 5*A2*5e-14*rho_crit*cosmo['Omega_m']/gz**2
    e1_TT =  Fact * (syy**2 - sxx**2) 
    return e1_TT

def Epsilon2_TT(cosmo,z,A2,rho_crit,sxy,sxx,syy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = - 5*A2*5e-14*rho_crit*cosmo['Omega_m']/gz**2
    e2_TT =  Fact * sxy*(sxx+syy)
    return e2_TT


In [4]:
#for i in np.arange(1)+1:
for i in np.arange(5)+1:

    catalog = Table.read('/global/cscratch1/sd/jharno/IA_Infusion/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i),
                     format='ascii')  # descriptions=['ra_deg', 'dec_deg', 'shear1', 'shear2', 'weight', 'z_true', 'delta', 's11', 's22', 's12'])
    s11=catalog['col8']
    s22=catalog['col9']
    s12=catalog['col10']
    z=catalog['col6']
    e1_NLA=Epsilon1_NLA(cosmo,z,Aia,rho_crit,s11,s22)
    e2_NLA=Epsilon2_NLA(cosmo,z,Aia,rho_crit,s12)

    e1_TT=Epsilon1_TT(cosmo,z,C2,rho_crit,s11,s22)
    e2_TT=Epsilon2_TT(cosmo,z,C2,rho_crit,s11,s22,s12)

    random1=np.random.normal(0, sigma_epsilon, e1_NLA.shape)
    random2=np.random.normal(0, sigma_epsilon, e2_NLA.shape)
    epsilon = Table([e1_NLA, e2_NLA, e1_TT, e2_TT, random1, random2], names = ('e1_NLA', 'e2_NLA', 'e1_TT', 'e2_TT', 'e1_noise', 'e2_noise'))
    epsilon.write('/global/cscratch1/sd/jharno/epsilon_{}.dat'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('/global/cscratch1/sd/jharno/epsilon_e1IAflip{}.dat'.format(i), format='ascii', overwrite=True)
    print('Done tomo',i)

Done tomo 1
Done tomo 2
Done tomo 3
Done tomo 4
Done tomo 5
